In [ ]:
from pathlib import Path

import IPython.display as ipd
import torch

from src.model import commons
from src.model.synthesizer import SynthesizerTrn
from src.params import Params
from src.text.convert import text_to_sequence
from src.utils.checkpoint import load_checkpoint

In [ ]:
def get_text(text: str, text_cleaners: list[str], language: str, phonemized: bool, stressed: bool):
    text_norm = text_to_sequence(text, text_cleaners, language, phonemized, stressed)
    text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)

    return text_norm

In [ ]:
params = Params.model_validate_json(Path("../files/configs/lt_studio_stressed_finetune.json").read_text(encoding="utf-8"))

net_g = SynthesizerTrn.from_params(params)
_ = net_g.eval()

In [ ]:
load_checkpoint(Path('../logs/studio_stressed_finetune/G_45000.pth'), net_g, None)

In [ ]:
# text = 'Penktadienio naktį daug kur trumpai palis'
# text = 'Gramatiškai veiksmažodis derinamas su veiksniu'
# text = 'mano vardas Gabija'
# text = 'Ar nori eiti su manimi drauge'
# text = 'Jis pastato savo mašina ir eina iki pastato galo.'
text = 'Jìs pastãto sàvo mašìna ir̃ eĩna ikì pãstato gãlo.'

stn_tst = get_text(
    text,
    params.data.text_cleaners,
    params.data.language,
    params.data.phonemized,
    params.data.stressed
)

with torch.inference_mode():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=0.667, noise_scale_w=0.5, length_scale=1.0)[0][0, 0]

ipd.display(ipd.Audio(audio.data.float().numpy(), rate=params.data.sampling_rate, normalize=False))